In [ ]:
pip install wikipedia

In [2]:
pip install textblob

  Using cached textblob-0.18.0.post0-py3-none-any.whl (626 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Using cached regex-2024.5.15-cp311-cp311-macosx_10_9_x86_64.whl (281 kB)

[notice] A new release of pip is available: 23.1.2 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gtts


[notice] A new release of pip is available: 23.1.2 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import spacy
from collections import Counter
from spacy.tokens import Span
import re
import random
import wikipedia
import textblob
from textblob import TextBlob
import gtts
from gtts import gTTS
import os

# Load the English language model
nlp = spacy.load("en_core_web_sm")

def extract_sentences(cv_file):
    # Read the CV file
    with open(cv_file, 'r') as file:
        cv_text = file.read()

    # Split the CV text into sentences
    sentences = re.split(r'[.!?]+', cv_text)
    return sentences

def extract_entities(sentences):
    # Extract named entities and their types from the sentences
    all_entities = []
    all_entity_labels = []
    for sentence in sentences:
        doc = nlp(sentence)
        entities = [ent for ent in doc.ents]
        all_entities.extend(entities)
        all_entity_labels.extend([ent.label_ for ent in entities])

    # Count the most common entity types
    entity_counts = Counter(all_entity_labels)
    top_entity_types = [ent_type for ent_type, count in entity_counts.most_common(3)]

    return all_entities, top_entity_types

def identify_skills(sentences):
    # Extract skills from the sentences
    skills = []
    for sentence in sentences:
        doc = nlp(sentence)
        skills.extend([token.text for token in doc if not token.is_stop and not token.is_punct and token.pos_ == "NOUN"])

    # Count the most common skills
    skill_counts = Counter(skills)
    top_skills = [skill for skill, count in skill_counts.most_common(5)]

    return top_skills

def summarize_cv(sentences):
    # Summarize the CV content
    summary = ""
    for sentence in sentences:
        doc = nlp(sentence)
        summary += " " + sentence

    return summary

def generate_questions(entities, top_entity_types, top_skills, summary):
    # Generate questions based on the extracted information
    questions = []
    follow_up_questions = []

    # Entity-based questions
    for ent_type in top_entity_types:
        ent_questions = [f"Can you tell me more about your experience with {ent.text} during your time at {ent.root.head.text}?" for ent in entities if ent.label_ == ent_type]
        questions.extend(ent_questions)

    # Skill-based questions
    for skill in top_skills:
        skill_question = f"How have you utilized your {skill} skills in your previous roles?"
        questions.append(skill_question)

    # Summarization-based questions
    summary_question = f"I noticed the summary mentioned {summary}. Can you elaborate on your specific contributions in that area?"
    questions.append(summary_question)

    # Follow-up questions
    follow_up_questions = [
        "Can you provide more details on your achievements in that role?",
        "How did you overcome any challenges you faced while working on that project?",
        "What was your specific contribution to the team's success in that endeavor?"
    ]

    return questions, follow_up_questions

def analyze_answers(sentences, answers):
    # Analyze the answers for factual information, general knowledge, and harsh/non-serious responses
    is_factual_cv = False
    is_general_knowledge = False
    is_harsh_or_non_serious = False

    # Implement your answer analysis logic here
    # ...

    return is_factual_cv, is_general_knowledge, is_harsh_or_non_serious

def conduct_interview(questions, follow_up_questions):
    score = 100

    print("Interview started. Please answer the following questions:")

    for i, question in enumerate(questions):
        tts = gTTS(text=f"Question {i+1}: {question}", lang='en')
        tts.save(f"question{i+1}.mp3")
        os.system(f"mpg123 question{i+1}.mp3")
        answer = input(f"Answer {i+1}: ")
        print(f"You answered: {answer}")
        is_factual_cv, is_general_knowledge, is_harsh_or_non_serious = analyze_answers(sentences, [answer])
        if is_harsh_or_non_serious:
            score -= 10  # Decrease score by 10 for a harsh or non-serious response
        if score <= 0:
            tts = gTTS(text="Interview selection result: Not selected", lang='en')
            tts.save("result.mp3")
            os.system("mpg123 result.mp3")
            return
        if is_factual_cv and is_general_knowledge:
            tts = gTTS(text="Interview selection result: Selected", lang='en')
            tts.save("result.mp3")
            os.system("mpg123 result.mp3")
            break

    print("\nFollow-up Questions:")
    for i, follow_up_question in enumerate(follow_up_questions):
        tts = gTTS(text=f"Follow-up Question {i+1}: {follow_up_question}", lang='en')
        tts.save(f"follow_up{i+1}.mp3")
        os.system(f"mpg123 follow_up{i+1}.mp3")
        answer = input(f"Answer {i+1}: ")
        print(f"You answered: {answer}")
        is_factual_cv, is_general_knowledge, is_harsh_or_non_serious = analyze_answers(sentences, [answer])
        if is_harsh_or_non_serious:
            score -= 10  # Decrease score by 10 for a harsh or non-serious response
        if score <= 0:
            tts = gTTS(text="Interview selection result: Not selected", lang='en')
            tts.save("result.mp3")
            os.system("mpg123 result.mp3")
            return
        if is_factual_cv and is_general_knowledge:
            tts = gTTS(text="Interview selection result: Selected", lang='en')
            tts.save("result.mp3")
            os.system("mpg123 result.mp3")
            break

    if score > 0:
        tts = gTTS(text=f"Your final score is: {score}", lang='en')
        tts.save("result.mp3")
        os.system("mpg123 result.mp3")
    else:
        tts = gTTS(text="Interview selection result: Not selected", lang='en')
        tts.save("result.mp3")
        os.system("mpg123 result.mp3")

# Example usage
cv_file = "cv.txt"
sentences = extract_sentences(cv_file)
entities, top_entity_types = extract_entities(sentences)
top_skills = identify_skills(sentences)
summary = summarize_cv(sentences)
questions, follow_up_questions = generate_questions(entities, top_entity_types, top_skills, summary)

conduct_interview(questions, follow_up_questions)


Interview started. Please answer the following questions:


High Performance MPEG 1.0/2.0/2.5 Audio Player for Layers 1, 2 and 3
	version 1.32.3; written and copyright by Michael Hipp and others
	free software (LGPL) without any warranty but with best wishes

Playing MPEG stream 1 of 1: question1.mp3 ...

MPEG 2.0 L III cbr64 24000 mono

[0:09] Decoding of question1.mp3 finished.


You answered: It was good, I got to learn so many things which involved programming languages and projects using sql and python


High Performance MPEG 1.0/2.0/2.5 Audio Player for Layers 1, 2 and 3
	version 1.32.3; written and copyright by Michael Hipp and others
	free software (LGPL) without any warranty but with best wishes

Playing MPEG stream 1 of 1: question2.mp3 ...

MPEG 2.0 L III cbr64 24000 mono

[0:07] Decoding of question2.mp3 finished.


You answered: I learned sql alongside python and used both of them in my recent project of web scraping which involved FLASK as well.


...

High Performance MPEG 1.0/2.0/2.5 Audio Player for Layers 1, 2 and 3
	version 1.32.3; written and copyright by Michael Hipp and others
	free software (LGPL) without any warranty but with best wishes

Playing MPEG stream 1 of 1: follow_up2.mp3 ...

MPEG 2.0 L III cbr64 24000 mono

[0:07] Decoding of follow_up2.mp3 finished.


You answered: My co-workers were a bit bossy, so I had to deal with a very smart way with them to make them do the best work


High Performance MPEG 1.0/2.0/2.5 Audio Player for Layers 1, 2 and 3
	version 1.32.3; written and copyright by Michael Hipp and others
	free software (LGPL) without any warranty but with best wishes

Playing MPEG stream 1 of 1: follow_up3.mp3 ...

MPEG 2.0 L III cbr64 24000 mono

[0:07] Decoding of follow_up3.mp3 finished.


You answered: Nothing special, it was just my patience and my professional attitude towards my work


High Performance MPEG 1.0/2.0/2.5 Audio Player for Layers 1, 2 and 3
	version 1.32.3; written and copyright by Michael Hipp and others
	free software (LGPL) without any warranty but with best wishes

Playing MPEG stream 1 of 1: result.mp3 ...

MPEG 2.0 L III cbr64 24000 mono

[0:03] Decoding of result.mp3 finished.
